# Nss Midcourse project: Creating pathfinder 2e monster encounter


notes about base builds

abilty scores
* All characters start with a 10 in stats
* ability ancester flaws = drops to 8
* ancester ability boosts = rises to 12

Misc/other categories
* column for 'situational bonus'-> things like darkvision
* Maybe have a bonus for speed? Like elves get a bonus because more than base speed, but dwarfs get a minus because less than base speed
* Some ancestries have equipment (like dwarf clan dagger)


In [3]:
#Import libraries 
import pandas as pd
import numpy as np
import requests
import matplotlib as plt


import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

## Try loading data from archives of nethys website


### Pull out all ancestry information 
dwarf= 'https://2e.aonprd.com/Ancestries.aspx?ID=59'
elf= https://2e.aonprd.com/Ancestries.aspx?ID=60
halfling= https://2e.aonprd.com/Ancestries.aspx?ID=63
human= https://2e.aonprd.com/Ancestries.aspx?ID=64
orc= https://2e.aonprd.com/Ancestries.aspx?ID=66

### Webscrapping use beautiful soup

In [79]:
URL = 'https://2e.aonprd.com/Ancestries.aspx?ID=59'

headers = {
    "User-Agent": "MyPythonScraper"
}

response = requests.get(URL, headers=headers)
response.status_code
assert response.status_code == 200

#convert it into a soup object
soup = BeautifulSoup(response.text)

### webscrapping pull out the information with the h2 class to create a list of relevant section titles 

In [266]:
### insepcting the datapage- all ancenstries are within a <h2 class> order

ancestry_title= soup.findAll('h2', attrs={"class":"title"})

ancestry_page_subtitles=[]
for section_label in ancestry_title:
    output_text= section_label.text
    ancestry_page_subtitles.append(output_text)

#A list of just the header subtitles
ancestry_page_subtitles

['You Might...',
 'Others Probably...',
 'Physical Description',
 'Society',
 'Beliefs',
 'Names',
 'Hit Points',
 'Size',
 'Speed',
 'Attribute Boosts',
 'Attribute Flaw',
 'Languages',
 'Darkvision',
 'Clan Dagger']

### Issue with the webpage formatting: the text contained between the header is not a flagged html code tag or attribute, so have to get creative on how to get the information 


In [194]:

load_line_order=[] 

for section in range(0, len(soup.findAll('h1'))):
    #loop through all the h1 class items to find the index for '{ancestry} Mechanics' section
    if soup.findAll('h1')[section].text.__contains__('Mechanics'):
        #Save the NEXT h1 section text so the loop will only pull out the information in the mechanics section
        end_line_flag= soup.findAll('h1')[section+1].text
        start_point= soup.findAll('h1')[section]
        output_by_line=[]
        #Create a while loop to pull out the informaiton and save the text info in the output list, but stop once it hits the next h1 section
        while line_text != end_line_flag:
            line_text= start_point.next_sibling
            output_by_line.append(line_text.text)
            start_point= line_text

<h2 class="title">Hit Points</h2>
10
<br/>
<h2 class="title">Size</h2>
Medium
<br/>
<h2 class="title">Speed</h2>
20 feet
<br/>
<h2 class="title">Attribute Boosts</h2>
Constitution
<br/>
Wisdom
<br/>
Free
<br/>
<h2 class="title">Attribute Flaw</h2>
Charisma
<br/>
<h2 class="title">Languages</h2>
<a href="Languages.aspx?ID=1">Common</a>
<br/>
<a href="Languages.aspx?ID=3">Dwarven</a>
<br/>
Additional languages equal to your Intelligence modifier (if it's positive). Choose from 
<a href="Languages.aspx?ID=5" style="text-decoration:underline">Gnomish</a>
, 
<a href="Languages.aspx?ID=6" style="text-decoration:underline">Goblin</a>
, 
<a href="Languages.aspx?ID=8" style="text-decoration:underline">Jotun</a>
, 
<a href="Languages.aspx?ID=9" style="text-decoration:underline">Orcish</a>
, 
<a href="Languages.aspx?ID=109" style="text-decoration:underline">Petran</a>
, 
<a href="Languages.aspx?ID=118" style="text-decoration:underline">Sakvroth</a>
, and any other languages to which you have acce

AttributeError: 'NoneType' object has no attribute 'text'

### There is an issue with the previous line of code that it throws an error instead of just breaking when the while condiiton is no longer met. BUT the code still creates and full output list, so I can use that for next filtering steps to get out ONLY the website text data that I care about

In [262]:
#create a list to search only for the terms that I care about 
information_flags= ['Hit Points', 'Speed', 'Attribute Boosts', 'Attribute Flaw', 'Darkvision']

#convert webscrapped information to a dataframe for easier cleaning 
output_df= pd.DataFrame(output_by_line, columns=['output'])
#remove any rows that are blank (from being HTML code)
output_df= output_df.replace('', 'nan').replace(',', 'nan')
output_df= output_df[output_df['output'] != 'nan']
#reset dataframe index to make using .iloc easiear
output_df= output_df.reset_index(drop=True)

In [264]:
#iterate over each row of the dataframe, and find the index value of ONLY the rows with the information of interst. Then save the index values to help grabbing the following informaiton easy

index_value_attributes=[] #save index as output list for future filtering
name_value_attributes=[] #save name of attribute to check filter
for text in range(0, len(output_df)):
    check_line= output_df.iloc[text]
    if check_line['output'] in information_flags:
        index_value_attributes.append(text)
        name_value_attributes.append(check_line['output'])

#Now for each attribute, grab all the information BETWEEN each index row 
## OMG i could use this method and use a list of all the h2 class headings as filters! Then this would be even more automated


[0, 4, 6, 10, 28]

In [268]:
name_value_attributes

['Hit Points', 'Speed', 'Attribute Boosts', 'Attribute Flaw', 'Darkvision']

In [269]:
index_value_attributes

[0, 4, 6, 10, 28]

**Current stopping point**: next task list
* Use the index values to navigate and take all the text in the FOLLOWING rows, and save these outputs in a dataframe. Make sure that the infomration_flags that have multiple rows preserve all the information in the rows (aka the data that is contained in multiple paragraph break lines on the website)
* Create a final statistics dataframe for each ancestry
* Iterate through each ancestry term on the webpage (based on the number index), and save them each as a row in the dataframe. Then save the 'ancestry stats' as a .csv file for future analysis
* Repeat for other webpage information

----

## pull information from rpgbot 
### This will create a term list of feats and equipment most reccomended to be useful for a build in a given ancestry. This will be used for any necessary optimization build decisions once full stats are created.

In [271]:
ancestry_name= 'dwarf'


URL = f"https://rpgbot.net/p2/characters/ancestries/{ancestry_name}/"

headers = {
    "User-Agent": "MyPythonScraper"
}

response = requests.get(URL, headers=headers)
response.status_code
assert response.status_code == 200

#convert it into a soup object
soup = BeautifulSoup(response.text)

#pull out a list of all the best feats to chose for the specific ancestry 
green_rating_repbot= soup.findAll('span', attrs={"class":"rating-green"})

output_green_choice= []
for item in green_rating_repbot:
    output_text = item.get_text()
    output_green_choice.append(output_text)

blue_rating_repbot= soup.findAll('span', attrs={"class":"rating-blue"})

output_blue_choice= []
for item in blue_rating_repbot:
    output_text = item.get_text()
    output_blue_choice.append(output_text)

#combine both lists to have a filter option 

best_choice_list= output_green_choice + output_blue_choice
best_choice_list

['Green',
 'Languages',
 'Anvil Dwarf',
 'Elemental Heart Dwarf',
 'Clan Pistol',
 'Clan’s Edge',
 'Dwarven Lore',
 'Forge-Days’ Rest',
 'Rock Runner',
 'Unburdened Iron',
 'Dwarven Weapon Cunning',
 'Protective Sheath',
 'Demolitionist',
 'Energy Blessed',
 'Stonewalker',
 'Dwarven Weapon Expertise',
 'Stonegate',
 'Blue',
 'Hit Points',
 'Ability Boosts',
 'Senses',
 'Forge Dwarf',
 'Strong-Blooded Dwarf',
 'Defy the Darkness',
 'Mountain’s Stoutness',
 'Stone Bones']